In [1]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import mutual_info_regression
import random
import platform
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from matplotlib import font_manager , rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Windows' :
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname = path).get_name()
    rc('font' , family = font_name)

In [2]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus 
import urllib 
import requests 
import json 
import pandas as pd

In [3]:
url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
params = '?' + urlencode({
    quote_plus("serviceKey"):"15crzzHwfhs3F6j4cXu4wyD%2F0gzkgLm9STKC2pBe9%2FnH%2FMrTBhBrwEJIwsQzXGTp4rTMQz53N72zPXl3uLg00Q%3D%3D",
    quote_plus("numOfRows"): "365",
    quote_plus("pageNo"): "1",
    quote_plus("dataType"): "JSON",
    quote_plus("dataCd") : "ASOS",
    quote_plus("dateCd") : "DAY",
    quote_plus("startDt"): "20190101",
    quote_plus("endDt"): "20191231",
    quote_plus("stnIds"): "279"
})
req = urllib.request.Request(url+unquote(params))

response_body = urlopen(req).read()
data = json.loads(response_body)

#print(data)



In [4]:
import json
import ujson

In [5]:
df = pd.DataFrame(data['response']['body']['items']['item'])

In [6]:
df.head()

,stnId,stnNm,tm,avgTa,minTa,minTaHrmt,maxTa,maxTaHrmt,mi10MaxRn,mi10MaxRnHrmt,...,avgM05Te,avgM10Te,avgM15Te,avgM30Te,avgM50Te,sumLrgEv,sumSmlEv,n99Rn,iscs,sumFogDur
0,279,구미,2019-01-01,-0.8,-3.8,126,2.4,1237,,,...,,,,,,,,,,
1,279,구미,2019-01-02,-1.8,-6.3,625,3.7,1416,,,...,,,,,,,,,,
2,279,구미,2019-01-03,-1.2,-6.3,341,5.2,1503,,,...,,,,,,,,,,
3,279,구미,2019-01-04,-0.2,-7.5,735,6.7,1338,,,...,,,,,,,,,,
4,279,구미,2019-01-05,0.5,-4.8,2345,5.9,1434,,,...,,,,,,,,,,


In [7]:
df = df.loc[:,['tm' , 'avgTa' , 'maxTa' , 'minTa' , 'avgWs'  , 'avgRhm' , 'avgPv','hr1MaxRn']]

In [8]:
df["tm"] = df["tm"].str.replace(pat=r'[^\w]', repl=r'', regex=True)

In [9]:
df.rename(columns={'tm': '날짜', 'avgTa': '평균기온' , 'maxTa' : '최고온도' , 'avgWs' : '평균풍속' , 'minTa' : '최저온도' , 'avgRhm' : '평균습도' , 'avgPv' : '평균기압'}, inplace=True)
df.to_csv("2019weather-1.csv" , encoding = 'euc-kr')

In [10]:
AccidentData = pd.read_csv('AccidentInfo.csv', encoding = 'euc-kr')
AccidentData.head()
AccidentData = AccidentData.loc[:,['사고번호' ,'도로명','사고내용' ,'기상상태']]
AccidentData.loc[:,'사고번호'] = AccidentData.loc[:,'사고번호']//100000000
AccidentData.head()

,사고번호,도로명,사고내용,기상상태
0,20191031,구미중앙로,중상사고,맑음
1,20190516,구미중앙로,경상사고,맑음
2,20190218,경부고속도로,중상사고,맑음
3,20191111,낙동강변로,경상사고,맑음
4,20190531,구미중앙로,부상신고사고,맑음


In [11]:
roadData = pd.read_csv('경상북도_구미시_가로수길정보.csv' , encoding = 'euc-kr')
roadData.head()
roadData = roadData.loc[:,[ '가로수길시작위도','가로수길시작경도','가로수길종료위도','가로수길종료경도','도로명']]
roadData.head()

,가로수길시작위도,가로수길시작경도,가로수길종료위도,가로수길종료경도,도로명
0,36.086722,128.361282,36.123435,128.362869,구미대로
1,36.117772,128.362639,36.118492,128.373445,구미대로 30길
2,36.070873,128.348511,36.086722,128.361282,금오대로
3,36.083575,128.389847,36.138013,128.451643,산호대로
4,36.136446,128.398603,36.141185,128.395483,대학로


In [12]:
AR_OUTER_JOIN = pd.merge(AccidentData, roadData, left_on='도로명', right_on='도로명', how='inner')

In [13]:
df = pd.read_csv('2019weather-1.csv' , encoding = 'euc-kr')

In [14]:
ARW_JOIN = pd.merge(AR_OUTER_JOIN, df, left_on='사고번호', right_on='날짜', how='inner')

In [15]:
ARW_JOIN.head()

,사고번호,도로명,사고내용,기상상태,가로수길시작위도,가로수길시작경도,가로수길종료위도,가로수길종료경도,Unnamed: 0,날짜,평균기온,최고온도,최저온도,평균풍속,평균습도,평균기압,hr1MaxRn
0,20191031,구미중앙로,중상사고,맑음,36.132378,128.323464,36.118854,128.362618,303,20191031,12.7,21.0,5.5,0.7,80.6,11.4,NaN
1,20191031,문장로,경상사고,맑음,36.135922,128.332311,36.153781,128.328704,303,20191031,12.7,21.0,5.5,0.7,80.6,11.4,NaN
2,20191031,박정희로,경상사고,맑음,36.078578,128.355424,36.124974,128.337068,303,20191031,12.7,21.0,5.5,0.7,80.6,11.4,NaN
3,20191031,옥계2공단로,중상사고,맑음,36.110903,128.417441,36.147214,128.418521,303,20191031,12.7,21.0,5.5,0.7,80.6,11.4,NaN
4,20191031,해마루공원로,경상사고,맑음,36.133301,128.428911,36.154072,128.435086,303,20191031,12.7,21.0,5.5,0.7,80.6,11.4,NaN


In [16]:
ARW_JOIN.drop('Unnamed: 0', axis=1, inplace=True)

In [17]:
ARW_JOIN.drop('날짜' , axis = 1 , inplace = True)

In [18]:
#ARW_JOIN.drop('도로명' , axis = 1 , inplace = True)

In [19]:
from sklearn import preprocessing
label_encoder=preprocessing.LabelEncoder()
ARW_JOIN['사고내용']=label_encoder.fit_transform(ARW_JOIN['사고내용'])

In [20]:
print('인코딩 클래스:',label_encoder.classes_)

인코딩 클래스: ['경상사고' '부상신고사고' '사망사고' '중상사고']


In [21]:
ARW_JOIN['도로명']=label_encoder.fit_transform(ARW_JOIN['도로명'])

In [22]:
print('인코딩 클래스:',label_encoder.classes_)

인코딩 클래스: ['1공단로' '1공단로3길' '1공단로4길' '1공단로6길' '1공단로9길' '3공단1로' '3공단2로' '3공단3로' '4공단로'
 '4공단로7길' '강동로' '검성로' '교리길' '구미대로' '구미중앙로' '구미중앙로45길' '금오대로' '금오산로'
 '금오산로22길' '낙동강변로' '남구미로' '단계동길' '대학로' '도봉로' '도봉로1길' '도봉로5길' '도안로' '들성로'
 '문장로' '박정희로' '백산로' '봉곡남로' '봉곡동로' '봉곡로' '봉곡서로' '비산로' '산업로' '산촌옥관로' '산호대로'
 '산호대로23길' '산호대로25길' '산호대로29길' '산호대로31길' '산호대로35길' '산호대로41길' '상림로' '상모로'
 '상사동로' '상사동로12길' '상사동로22길' '상사서로' '상사서로12길' '상사서로22길' '상사서로4길' '새마을로'
 '선기로' '선산대로' '선산중앙로' '선산중앙로7길' '선상서로' '송동로' '송선로' '송원동로' '송원서로' '송정대로'
 '수출대로' '수출대로3길' '수출대로7길' '신비로' '신시로' '신시로10길' '야은로' '야은로13길' '여헌로'
 '옥계2공단로' '옥계남로' '옥계북로' '왕산로' '원남로' '이계남로' '이계북로' '인동36길' '인동가산로' '조남로'
 '천생산길' '첨단기업1로' '첨단기업2로' '첨단기업5로' '칠성로' '해마루공원로' '형곡동로' '형곡로' '형곡서로'
 '형곡중앙로' '화신로' '흥안로']


In [23]:
ARW_JOIN.drop(['기상상태' , '가로수길시작위도' , '가로수길시작경도' , '가로수길종료위도' , '가로수길종료경도'] , axis = 1 , inplace = True)

In [24]:
ARW_JOIN.rename(columns={'hr1MaxRn' : '한 시간 최다 강수량'}, inplace=True)

In [25]:
ARW_JOIN = ARW_JOIN.fillna(0)

In [26]:
ARW_JOIN.head(10)

,사고번호,도로명,사고내용,평균기온,최고온도,최저온도,평균풍속,평균습도,평균기압,한 시간 최다 강수량
0,20191031,14,3,12.7,21.0,5.5,0.7,80.6,11.4,0.0
1,20191031,28,0,12.7,21.0,5.5,0.7,80.6,11.4,0.0
2,20191031,29,0,12.7,21.0,5.5,0.7,80.6,11.4,0.0
3,20191031,74,3,12.7,21.0,5.5,0.7,80.6,11.4,0.0
4,20191031,89,0,12.7,21.0,5.5,0.7,80.6,11.4,0.0
5,20190516,14,0,23.2,30.4,16.5,0.8,59.6,15.8,0.0
6,20190516,14,0,23.2,30.4,16.5,0.8,59.6,15.8,0.0
7,20190516,63,3,23.2,30.4,16.5,0.8,59.6,15.8,0.0
8,20190516,71,0,23.2,30.4,16.5,0.8,59.6,15.8,0.0
9,20190516,73,0,23.2,30.4,16.5,0.8,59.6,15.8,0.0


In [27]:
ARW_JOIN.to_csv("final.csv" , encoding  ='euc-kr')